In [2]:
from catboost import CatBoostClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, plot_roc_curve, make_scorer, f1_score, roc_auc_score, det_curve
from sklearn import preprocessing
from scipy import stats
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate, LeaveOneGroupOut, PredefinedSplit, GridSearchCV
import matplotlib.pyplot as plt
import os
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import CategoricalNB
import json

%matplotlib inline

In [3]:
def concat_dataframes(path, df_type):
    dfs_list = []
    dfs_rows_len_list = []
    
    for user in os.listdir(path):
        for file in os.listdir(os.path.join(path, user)):
            if file.find(df_type) != -1:
                df = pd.read_csv(os.path.join(path, user, file))
                
                if df_type != 'broadcasts':
                    df = df.drop(["timestamp"], axis=1)
#                 df = (df - df.min()) / (df.max() - df.min())
                
                df["user"] = int(user.split('_')[1])
                
                dfs_list.append(df)
    
    return pd.concat(dfs_list, ignore_index=True)


def get_dataframe(path, data_type, window_type, window_size):
    return concat_dataframes(os.path.join(path, window_type, window_size), data_type), create_file_for_results(data_type)

In [ ]:
DATA_PATH = '..\\..\\scripts\\_features_all'

DATA_TYPE = "bt"

WINDOW_TYPE = "rolling"
WINDOW_SIZE = "60s"

In [ ]:
df, _ = get_dataframe(DATA_PATH, DATA_TYPE, WINDOW_TYPE, WINDOW_SIZE)
features = df.columns.to_list()

In [ ]:
df, _ = process_train_df(df, features)

In [ ]:
fig = plt.figure(figsize=(25, 25))
plt.matshow(df.corr(), fignum=fig.number)
plt.xticks(range(df.shape[1]), df.columns, fontsize=18, rotation=90)
plt.yticks(range(df.shape[1]), df.columns, fontsize=18)
cb = plt.colorbar()
cb.ax.tick_params(labelsize=16)
plt.title("Correlation matrix", fontsize=20, y=-0.03)

In [ ]:
# sample = df.sample(10000)
# plt.figure(figsize=(16, 10))
# plt.scatter(x=sample['speed_mean'], y=sample['speed_var'], alpha=0.5, c=sample.user, cmap='magma')
# plt.colorbar()
# plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.font_manager
from sklearn import svm

xx, yy = np.meshgrid(np.linspace(-5, 5, 500), np.linspace(-5, 5, 500))
# Generate train data
X = 0.3 * np.random.randn(100, 2)
X_train = np.r_[X + 2, X - 2]
# Generate some regular novel observations
X = 0.3 * np.random.randn(20, 2)
X_test = np.r_[X + 2, X - 2]
# Generate some abnormal novel observations
X_outliers = np.random.uniform(low=-4, high=4, size=(20, 2))

# fit the model
clf = svm.OneClassSVM(nu=0.05, kernel="rbf", gamma=1)
clf.fit(X_train)
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)
y_pred_outliers = clf.predict(X_outliers)
n_error_train = y_pred_train[y_pred_train == -1].size
n_error_test = y_pred_test[y_pred_test == -1].size
n_error_outliers = y_pred_outliers[y_pred_outliers == 1].size

# plot the line, the points, and the nearest vectors to the plane
Z = clf.decision_function(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

plt.title("Novelty Detection")
plt.contourf(xx, yy, Z, levels=np.linspace(Z.min(), 0, 7), cmap=plt.cm.PuBu)
a = plt.contour(xx, yy, Z, levels=[0], linewidths=2, colors='darkred')
plt.contourf(xx, yy, Z, levels=[0, Z.max()], colors='palevioletred')

s = 40
b1 = plt.scatter(X_train[:, 0], X_train[:, 1], c='white', s=s, edgecolors='k')
b2 = plt.scatter(X_test[:, 0], X_test[:, 1], c='blueviolet', s=s,
                 edgecolors='k')
c = plt.scatter(X_outliers[:, 0], X_outliers[:, 1], c='gold', s=s,
                edgecolors='k')
plt.axis('tight')
plt.xlim((-5, 5))
plt.ylim((-5, 5))
plt.legend([a.collections[0], b1, b2, c],
           ["learned frontier", "training observations",
            "new regular observations", "new abnormal observations"],
           loc="upper left",
           prop=matplotlib.font_manager.FontProperties(size=11))
plt.xlabel(
    "error train: %d/200 ; errors novel regular: %d/40 ; "
    "errors novel abnormal: %d/40"
    % (n_error_train, n_error_test, n_error_outliers))
plt.show()

In [ ]:
# df = (df - df.min()) / (df.max() - df.min())

In [ ]:
# fig = plt.figure(figsize=(20, 25))
# plt.matshow(df.corr(), fignum=fig.number)
# plt.xticks(range(df.shape[1]), df.columns, fontsize=18, rotation=90)
# plt.yticks(range(df.shape[1]), df.columns, fontsize=18)
# cb = plt.colorbar()
# cb.ax.tick_params(labelsize=16)
# plt.title("Correlation matrix", fontsize=20, y=-0.03)

In [ ]:
# sample = df.sample(1000)
# plt.figure(figsize=(16, 10))
# plt.scatter(x=sample['conn_level_mean'], y=sample['count_var'], alpha=0.5, c=sample.user, cmap='magma')
# plt.colorbar()
# plt.show()

In [ ]:
# for user in df.user.unique():
#     for valid_user in df.user.unique():
#         if user != valid_user:
#             print('---------------------------------------------------------------------------')
#             print('Valid user: ', valid_user, 'Extracted user: ', user)
#             print('---------------------------------------------------------------------------')
#             df1, df_ = extract_delayed_user(df.copy(), user)
#             df1['user'] = 0
#             df_ = split_users_into_two_classes(df_.copy(), valid_user)
#             df_ = resample(df_)

#             dataset = df_.to_numpy()
#             X = dataset[:, :-1]
#             y = dataset[:, -1]

#             X_test = df1.to_numpy()[:, :-1]
#             y_test = df1.to_numpy()[:, -1]
            
#             model = CatBoostClassifier(iterations=100, depth=6, loss_function='Logloss')
#             model.fit(X, y, verbose=False)

#             preds_class = model.predict(X_test)
#             print('Accuracy: ', accuracy_score(preds_class, y_test))

#             sum_ = 0
#             imp = [ (x, i) for x, i in zip(model.feature_importances_, range(len(model.feature_importances_)) )]
#             sorted_ = sorted(imp, key=lambda tup: tup[0])
#             for i in range(len(sorted_)):
#                 if sorted_[i][0] > 5:
#                     print(sorted_[i][1], ': ', df_.columns[sorted_[i][1]], ' - ', sorted_[i][0])

#             print('---------------------------------------------------------------------------')
#             print('---------------------------------------------------------------------------')

In [ ]:
# for d in [VALIDATION_CATBOOST_BIG_DICT, VALIDATION_RFC_BIG_DICT, \
#           VALIDATION_SVC_BIG_DICT, VALIDATION_LR_BIG_DICT]:
    
#     for user, res in d.items():
#         print("Valid User: ", user)
#         print("--------------------------------------------------------------------------------")
#         means_acc = []
#         means_prec = []
#         means_rec = []
#         means_roc = []
#         means_f1 = []
        
#         for ex_user, ex_res in res.items():
#             print('Ex user: ', ex_user)
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

#             y_true = ex_res['y_test']
#             y_pred = ex_res['y_predict']
#             if len(ex_res['y_proba'].shape) > 1 and ex_res['y_proba'].shape[1] > 1:
#                 y_proba = ex_res['y_proba'][:, 1]
#             else:
#                 y_proba = ex_res['y_proba']
                
#             acc = accuracy_score(y_true, y_pred)
#             prec = precision_score(y_true, y_pred)
#             rec = recall_score(y_true, y_pred)
#             f1 = f1_score(y_true, y_pred)
            
#             means_acc.append(acc)
#             means_prec.append(prec)
#             means_rec.append(rec)
#             means_f1.append(f1)

#             print('Accuracy: ', acc)
#             print('Precision: ', prec)
#             print('Recall: ', rec)
#             try:
#                 roc = roc_auc_score(y_true, y_proba)
#                 means_roc.append(roc)
#                 print('ROC-AUC: ', roc)
#             except Exception as e:
#                 print('ROC-AUC: skip')
#             print('F1: ', f1)
#             print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
            
#         print('Mean accuracy: ', sum(means_acc) / len(means_acc))
#         print('mean Precision: ', sum(means_prec) / len(means_prec))
#         print('mean Recall: ', sum(means_rec) / len(means_rec))
#         if len(means_roc) > 0:
#             print('mean ROC-AUC: ', sum(means_roc) / len(means_roc))
#         print('mean F1: ', sum(means_f1) / len(means_f1))

#         print("--------------------------------------------------------------------------------")

In [ ]:
# gs_df_parts = []
# for user in df.labels.unique():
#     new_df = df[df.labels == user].sample(int(df[df.labels == user].shape[0] * 0.2)).copy()
#     gs_df_parts.append(new_df)

# df = pd.concat(gs_df_parts)

In [ ]:
# sample = df.sample(10000)

# fig, ax = plt.subplots(figsize=(12, 9))

# scatter = ax.scatter(x=sample['disappeared_devices_count_mean_mad'], y=sample['jaccard_index_var_skew'], \
#                      alpha=0.5, c=sample.user, cmap='plasma')

# # produce a legend with the unique colors from the scatter
# plt.rcParams['legend.title_fontsize'] = 'x-large'
# legend1 = ax.legend(*scatter.legend_elements(), loc="upper right", title="Users", fontsize=14)
# ax.add_artist(legend1)

# plt.xlabel('MAD среднего числа исчезнувших устройств', fontsize=12)
# plt.ylabel('Skew дисперсии расстояния Жаккара', fontsize=12)

# plt.savefig('.\\after.png', dpi=500)
# plt.show()

In [ ]:
# C = 1
# kernel = 'rbf'
# degree = 1
# gamma = 5

# user = 8

# df['labels'] = df['user']

# df_ = resample(df.copy())
# df_ = split_users_into_two_classes(df_.copy(), user)
# df_ = resample(df_)
    
# df_.loc[df_.user == 0, 'user'] = -1
    
# df_ = df_.drop('labels', axis=1)
    
# model = SVC(C=C, kernel=kernel, degree=degree, gamma=gamma)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import ListedColormap
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler
# from sklearn.datasets import make_moons, make_circles, make_classification
# from sklearn.neural_network import MLPClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC
# from sklearn.gaussian_process import GaussianProcessClassifier
# from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

# h = .02  # step size in the mesh

# figure = plt.figure(figsize=(27, 9))
# i = 1

# ds = df_.to_numpy().copy()
# np.random.shuffle(ds)

# X = ds[:, :-1]
# y = ds[:, -1]

# X_train = X[:10000, [14, 31]]
# y_train = y[:10000]

# X_test = X[160000:, [14, 31]]
# y_test = y[160000:]

# x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
# y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5

# xx, yy = np.meshgrid(
#                     np.arange(x_min, x_max, h),
#                     np.arange(y_min, y_max, h))

# cm = plt.cm.RdBu
# cm_bright = ListedColormap(['#FF0000', '#0000FF'])
# ax = plt.subplot(1, 2, 1)
# ax.set_title("Input data")
# ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
#                edgecolors='k')
# ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright, alpha=0.6,
#             edgecolors='k')
# ax.set_xlim(xx.min(), xx.max())
# ax.set_ylim(yy.min(), yy.max())
# ax.set_xticks(())
# ax.set_yticks(())

# ax = plt.subplot(1, 1 + 1, 1)
# model.fit(X_train, y_train)
# score = model.score(X_test, y_test)

# if hasattr(model, "decision_function"):
#     Z = model.decision_function(np.c_[xx.ravel(), yy.ravel()])
# else:
#     Z = model.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1]

# Z = Z.reshape(xx.shape)
# ax.contourf(xx, yy, Z, cmap=cm, alpha=.4)

# ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=cm_bright,
#                    edgecolors='k')
# ax.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm_bright,
#                    edgecolors='k', alpha=0.6)

# ax.set_xlim(xx.min(), xx.max())
# ax.set_ylim(yy.min(), yy.max())
# ax.set_xticks(())
# ax.set_yticks(())
# # ax.set_title(name)
# ax.text(xx.max() - .3, yy.min() + .3, ('%.2f' % score).lstrip('0'),
#                 size=15, horizontalalignment='right')

# plt.tight_layout()
# plt.show()

In [ ]:
# fpr, tpr, threshold = roc_curve(y_true, y_proba)
# roc_auc = auc(fpr, tpr)

# plt.title('ROC-curve')
# plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
# plt.legend(loc = 'lower right')
# plt.plot([0, 1], [0, 1],'r--')
# plt.xlim([0, 1])
# plt.ylim([0, 1])
# plt.ylabel('True Positive Rate')
# plt.xlabel('False Positive Rate')
# plt.savefig("..\\")